# PyWren RISECamp, 2017

## Data Analytics with PyWren

In this section, we will use PyWren explore a dataset of Wikipedia records.

## 0. The Data

We've prepared an S3 bucket with 20GB of Wikipedia traffic statistics data obtained from http://aws.amazon.com/datasets/4182. To make the analysis more feasible for the short time you're here, we've shortened the dataset to three days worth of data (May 5 to May 7, 2009; roughly 20G and 329 million entries). 

Let's take a look into the bucket with our dataset. We'll print a few files from a few files from our bucket.


In [ ]:
from training import wikipedia_bucket, list_keys_with_prefix, read_from_s3

filenames = list_keys_with_prefix(wikipedia_bucket, "wikistats_20090505_restricted-01/")
for filename in filenames[:20]:
    print(filename)

There are 74 files (2 of which are intentionally left empty). Each file consists of a list of records. Let's go take a look into the first file. This may take a while.

In [ ]:
# Read a file into memory, and split on newlines.
records = read_from_s3(wikipedia_bucket, "wikistats_20090505_restricted-01/part-00001").split("\n")

print("The total number of records in this file is {}, but here are the first 20".format(len(records)))
for i in range(20):
    print(records[i])

Each record has stats for a single Wikipedia page. The schema is:

`<date_time> <project_code> <page_name> <page_views> <page_size>`

- `<date_time>` specifies a date in YYYYMMDD-HHmmSS format (year, month, day, hour minute, second).
- `<project_code>` specifies the language the page is written in.
- `<page_title>` gives the page title.
- `<page_views>` gives the number of page views in the hour-long time slot starting at `<data_time>`. 
- `<page_size>` gives the size in bytes of the  page.

Now that we have a better understanding of the structure of our data, we can start running some interesting queries. 

Because the data are so large, it takes us quite a while to load just one file, as we saw in the previous exercise. You could imagine it would take way too long to process all of the data sequentially. Thankfully, we have [PyWren](https://www.youtube.com/watch?v=VSaDPc1Cs5U).

## 1. To the ~~Batmobile~~ Cloud!!!!!!

With PyWren, we can invoke lambdas to do everything in parallel in the cloud, including reading and writing files from S3. First we'll again set up an executor.

In [ ]:
import pywren
pwex = pywren.default_executor()

## 2. Count
Let’s see how many records in total are in this data set.

***Exercise:*** modify the code below to count the number of records in our dataset.

In [ ]:
filenames = list_keys_with_prefix(wikipedia_bucket, "wikistats_20090505_restricted-01/")

def count(filename):
    data = read_from_s3(wikipedia_bucket, filename)
    # fill me in:
    return 

print ("invoking pywren jobs...")
futures = pwex.map(count, filenames)
print ("working...")
pywren.wait(futures)

result = sum([f.result() for f in futures])
print(result)

This should launch 73 PyWren tasks. After finishing the job, let's plot again to check the execution. This should look more interesting than the simple job before.

In [ ]:
from training import plot_pywren_execution
plot_pywren_execution(futures)

# 3. Visits for English Pages
Now let’s calculate the total view count on all English pages for each date in our dataset. From each record we want to extract three pieces of information: the access date, the view count, and language. 
Recall from above that the second field of a record is the `project code` For example, the project code `en` indicates an English page.

With PyWren, we can `map` over every file with a function that will parse through the records in the file and return the sum of the view counts of each English page. After the jobs return, we can simply aggregate the page count sum for each file and add them together.

In [ ]:
from itertools import groupby
from operator import itemgetter
from functools import reduce

def aggregate_count(key_value_list):
    
    def reduce_f(obj1, obj2):
        return(obj1[0], obj1[1] + obj2[1])
    
    counts = [reduce(reduce_f, group) for _, group 
          in groupby(sorted(key_value_list), key=itemgetter(0))]
    return counts

def english_page_count(filename):
    data = read_from_s3(wikipedia_bucket, filename)
    
    # return True if a record corresponds to an english page.
    def is_page_english(record):
        # Fix me.
        return False
    
    # filter out the english pages
    en_pages = filter(is_page_english, data.split("\n"))
    
    # projection to create (date, pagecount) pairs
    def make_date_viewcount_pair(record):
        split_record = record.split()
        
        # the daate is the first entry of the record.
        # We only want the YYYYMMDD characters.
        date = split_record[0][:8]
        
        view_count = int(split_record[3])
        return (date, view_count)
    
    en_kvpair_list = [make_date_viewcount_pair(p) for p in en_pages]

    return aggregate_count(en_kvpair_list)
    
futures = pwex.map(english_page_count, filenames)
pywren.wait(futures)

results = [f.result() for f in futures]
en_page_counts_by_date = aggregate_count([x for y in results for x in y])
print(en_page_counts_by_date)

# 4. Bonus: Finding the 15 most popular pages.
Suppose we wanted to find the 15 most popular pages in the dataset. One immediate, "naive" solution would be to `map` a function that takes in a filename and returns the `(page_views, title)` pair for each record in the file, and then find the 15 most viewed pages from the returned results. This approach however, involves us iterating through some 51 million records, when we ideally could do some more work in parallel.

To offload more work to the cloud, we could change our `map` function to return the 15 most popular records in each file. This would reduce the amount of work we do on our single node.

In [ ]:
import pywren
pwex = pywren.default_executor()

# Takes a list of (size, page_title) pairs
# and returns the 15 largest kv pairs.
def fifteen_biggest(list_of_pairs):
    # fill me in.
    # Hint. You can use a min heap. https://docs.python.org/2/library/heapq.html
    return []
    
def process_file(filename):
    data = read_from_s3(wikipedia_bucket, filename)
    data = data.split("\n")
    data = filter(lambda x: len(x) > 4, data)
    
    # Takes a single record and returns a tuple of (page_size, page_title)
    # Page_size should be an int
    def extract_kv_pair(record):
        record = record.split(" ")
        # Fix me.
        return (None, None)
    data = map(extract_kv_pair, data)
    return fifteen_biggest(data)

futures = pwex.map(process_file, filenames)
results = pwex.get_all_results(futures)

flattened_list = []
for result in results:
    for elem in result:
        flattened_list.append(elem)

final_list = fifteen_biggest(flattened_list)
for item in final_list:
    print(item)
